<a href="https://colab.research.google.com/github/Hyeon-Kang/NLP/blob/master/week03_Naive_Bayes_model/NB_Grid%20search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# 20개의 뉴스 분야 카테고리를 얼마나 잘 분류하는가?

from sklearn.datasets import fetch_20newsgroups

twenty_train = fetch_20newsgroups(subset = 'train', shuffle=True)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

# 문서 집합에서 단어토큰 생성, 각 단어의 수를 세어 bag of word(순서 상관없음)로 인코딩한 벡터로 만들어줌

count_vect = CountVectorizer() # 추출한 단어마다 고유 인덱스를 부여하여 벡터로 표현함
X_train_counts = count_vect.fit_transform(twenty_train.data) 
print('Count Vector : ', X_train_counts.shape) # 출력 : (문서 개수, 단어 개수)

Count Vector :  (11314, 130107)


In [0]:
from sklearn.feature_extraction.text import TfidfTransformer

# TF : Term Frequency - 한 문서 내에서 특정 단어가 사용된 빈도
# IDF : Inverse Document Frequency - 특정 단어가 등장하는 문서의 빈도
# 정보 검색과 데이터 마이닝에서 이용하는 가중치

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print('Tfidf : ', X_train_tfidf.shape)

Tfidf :  (11314, 130107)


In [0]:
# Naive Bayes (다항분포/분류 알고리즘)
print(' ---- Naive Bayes algorithm ---- ') 

from sklearn.naive_bayes import MultinomialNB

# 분류 모델에 학습데이터와 해당 분야(label)를 학습시킴
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

from sklearn.pipeline import Pipeline

# pipeline을 통해 전처리 단계와 분류 모델을 합침
text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', MultinomialNB())])

# 위에서 작성한 pipeline 모델에 입력 데이터와 출력 결과(label)을 학습
text_clf = text_clf.fit(twenty_train.data, twenty_train.target)

import numpy as np

twenty_test = fetch_20newsgroups(subset = 'test', shuffle = True) 
predicted = text_clf.predict(twenty_test.data)  # 학습 모델에 test data를 집어넣어 성능 측정
print('Accuracy : ', np.mean(predicted == twenty_test.target))





 ---- Naive Bayes algorithm ---- 
Accuracy :  0.7738980350504514


In [0]:
# 입력한 여러 경우의 파라미터를 비교하여 최적의 파라미터를 찾는 방법
print(' ---- Grid serach : Naive Bayes algorithm ---- ')

#from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

# 이하 입력한 범위내의 파라미터를 모두 구해보고 최적의 경우를 도출함
parameters = {'vect__ngram_range' : [(1,1), (1,2)], #  (1,1) : 단어별 구분, (1,2) bigram 구분
             'tfidf__use_idf' : (True, False), # Tf-Idf 사용한 경우, 사용하지 않은 경우
             'clf__alpha' : (1e-2, 1e-3)} # 나이브베이즈 다항분류의 페널티 파라미터 값 0.01 or 0.001

gs_clf = GridSearchCV(text_clf, parameters, n_jobs = -1) # -1 : 모든 cpu 사용
gs_clf = gs_clf.fit(twenty_train.data, twenty_train.target)

print('Accuracy : ', gs_clf.best_score_)

# print(gs_clf.best_params_)
# print('Accuracy : ', np.mean(predicted == twenty_test.target))

 ---- Grid serach : Naive Bayes algorithm ---- 


c:\python\venv\tensorflow\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Accuracy :  0.9067526957751458
